Notebook containing tools to summarize AND interact with different content types (websites, pdfs, ppts, videos) <br>
The Notebook employs various python packages to extract required contents from URLs <br>
And employs an llm connector to connect to frontier llms (llama, mixtral, gemini, gpt etc - gpt-4o in this case) <br>
There are different sections for eacvh content type and it contains summary and qna - which can be run in Test section under each <br>

# Imports

In [1]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [ ]:
from src.llm_connector import create_model_client, chat_with_model

In [3]:
# Website Scraper
from bs4 import BeautifulSoup 

In [20]:
# PDF reader
# !pip install PyPDF2
from io import BytesIO
from PyPDF2 import PdfReader
from urllib.parse import urlparse

In [9]:
# PPT 
# !pip install python-pptx
from pptx import Presentation

In [150]:
# Youtube
# !pip install youtube-transcript-api
from youtube_transcript_api import YouTubeTranscriptApi
import re

# Universal Methods

In [ ]:
def get_model_response(model_name, user_message, system_message=None):
    print(f"Using model: {model_name}")
    if system_message:
        print(f"With system prompt: {system_message[:50]}...")
    if user_message:
        print(f"With user prompt: {user_message[:50]}...")
    try:
        response = chat_with_model(model_name, user_message, system_message)
        return response
    except Exception as e:
        print(f"Error: {e}")
        return None

# Test

In [ ]:

response = get_model_response(
    model_name="gpt-4o",
    user_message="What are the different types of clouds?",
    system_message="You are a helpful assistant with an expertise in meterology"
)

if response:
    print(response)

# Websites

## Scraper

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

## Summarize

In [6]:
system_prompt_website_summary = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
def get_user_prompt_website_summary(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
def summarize_website(url):
    website = Website(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_website_summary(website),
        system_message=system_prompt_website_summary
    )
    return response

### Test

In [9]:
def display_website_summary(url):
    summary = summarize_website(url)
    display(Markdown(summary))

In [10]:
display_website_summary("https://netflixtechblog.com/foundation-model-for-personalized-recommendation-1a0bd8e02d39")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled Foundation Mod...
Using OpenAI with model: gpt-4o


The article from the Netflix Technology Blog discusses the new foundation model for personalized recommendations being developed by Netflix's engineering team. As Netflix expands its personalization algorithms, the challenge has been in maintaining these models and transferring innovations across different systems. Inspired by the evolution from multiple small models to large language models in NLP, Netflix aims to create a centralized recommendation model. This model uses data-centric approaches and semi-supervised learning to process billions of user interactions effectively, applying techniques like interaction tokenization and sparse attention to enhance scalability and accuracy. The model also addresses unique challenges in recommendation systems, like entity cold-starting, by integrating metadata as well as user interactions. Its applications span predictive modeling and generating user and entity embeddings that can be used across various services, marking a significant shift from specialized models to a comprehensive system. The blog underscores Netflix's continued innovation in improving user experiences through advanced machine learning and AI techniques.

## QnA

In [11]:
system_prompt_website_qna = "You are an assistant that analyzes the contents of a website \
and answers questions related to the content of the website, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
def get_user_prompt_website_qna(website, question):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows \n\n"
    user_prompt += website.text
    user_prompt += f"\n\nPlease answer the following question related to this website: {question}"
    return user_prompt

In [13]:
def website_qna(url, question):
    website = Website(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_website_qna(website, question),
        system_message=system_prompt_website_qna
    )
    return response

### Test

In [14]:
def display_website_answer(url, question):
    answer = website_qna(url, question)
    display(Markdown(answer))

In [15]:
display_website_answer("https://netflixtechblog.com/foundation-model-for-personalized-recommendation-1a0bd8e02d39", "From a data scientist's perspective, what is this blog sayin? please explain in detail the model they're using, whats the approach, and how is this about LLM?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled Foundation Mod...
Using OpenAI with model: gpt-4o


The blog post from Netflix TechBlog discusses the development of a Foundation Model for Personalized Recommendations, drawing inspiration from the advancements in Large Language Models (LLMs). Let’s break down the key elements from a data scientist's perspective:

### Model and Approach

1. **Unified Recommender System**: Netflix is evolving from using multiple specialized machine learning models for different recommendation tasks (e.g., “Continue Watching,” “Today’s Top Picks for You”) to a centralized recommendation architecture. This design aims to improve efficiency, maintainability, and the ability to propagate innovations across different recommendation tasks.

2. **Foundation Model Concept**: Inspired by the paradigm shift in NLP towards large language models that can perform multiple tasks, Netflix is implementing a similar approach in recommendations. The foundational recommendation model aggregates and generalizes user interaction data, catering to various downstream recommendation tasks.

3. **Data-Centric Approach**: Similar to LLMs, the focus shifts from a model-centric approach (heavily reliant on feature engineering) to a data-centric one. Emphasis is placed on accumulating large-scale, high-quality data and employing end-to-end learning where practical.

4. **Tokenization of User Interactions**: User actions are tokenized to create meaningful sequences, preserving essential information like watch duration and engagement types. This process balances data granularity with sequence compression to keep training and inference feasible within real-world constraints.

5. **Model Architecture**:
   - **Sparse Attention and Sliding Window Sampling**: To handle the extensive interaction histories that surpass the typical context windows of recommendation models, techniques like sparse attention and sliding window sampling are used. These methods help process longer sequences efficiently.

6. **Training Objectives**:
    - **Autoregressive Next-Token Prediction**: This aligns with the GPT model's training objective and exploits large-scale unlabeled data.
    - Adjustments are made to account for the varying importance of different interactions, such as distinguishing between short trailer plays and full movie watches.
    - Multi-token prediction adjusts the focus to capture long-term dependencies more effectively than single-token prediction might.

7. **Cold Start Solutions**:
    - **Incremental Training**: Life-long learning capabilities, such as reusable parameters for new content, enable rapid adaptation as titles are added to the platform.

8. **Embeddings and Metadata**:
    - New titles incorporate metadata into their embeddings to provide reasonable starting points until user interaction data become available. The combined use of metadata and IDs address cold-start issues.

### Relationship to LLMs

Netflix applies insights from LLMs in the following ways:

- **Transformation from Specialized Models to Foundation Levels**: Following LLMs’ transformation, the idea is to move from discrete, specialized models to a single, versatile model.
- **Large-Scale Semi-Supervised Training**: Like LLMs, this model benefits from semi-supervised training on a wide array of unlabeled data to acquire a deep understanding of member preferences.
- **End-to-End Learning and Scaling**: The scaling law seen in LLMs for increased performance with larger models and data also applies to Netflix’s foundation model. The aim is to facilitate expansive data engagement and generate better recommendation solutions.

Overall, this blog highlights the technical nuances of transitioning to a more holistic, scalable recommendation system, leveraging ideas from LLM successes to enhance personalization capabilities across Netflix’s vast viewership and content catalog.

In [16]:
display_website_answer("https://netflixtechblog.com/foundation-model-for-personalized-recommendation-1a0bd8e02d39", "I don't understand, this is so high level -what is the difference from their previous approach, how can you tokenize user behaviour, what do you mean by Autoregressive Next-Token Prediction, how are metadata and embeddings handled? i'm a data scientist, but i still dont see a clear picture of what they're doing, explain clearly with an example please?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled Foundation Mod...
Using OpenAI with model: gpt-4o


To fully grasp Netflix's new approach to personalized recommendations, it's worth breaking it down component by component as they've shifted strategies compared to their previous systems:

### Previous Approach:
Netflix historically used multiple specialized models for different recommendation functions like "Continue Watching" or "Today's Top Picks." Each model operated largely independently, presenting challenges in sharing insights or leveraging innovations across models, especially as these were trained separately on similar datasets.

### New Approach:

#### 1. **Tokenization of User Behavior:**
   - **Tokenization** is analogous to how language models process sentences. For user interactions, not every action is of equal relevance (e.g., browsing vs. watching an entire movie). Netflix condenses sequences of interactions (like watching trailers, pausing, or finishing a movie) into more meaningful "tokens." This is similar to NLP's Byte Pair Encoding (BPE). For instance, two consecutive actions like "played episode" and "binge-watched season" might be merged into a "completed series" token, summarizing user behavior compactly.

#### 2. **Autoregressive Next-Token Prediction:**
   - **Autoregressive modeling** involves predicting the next item in a sequence using prior data. Here, after tokenizing user behaviors, the model predicts what the user will likely interact with next. Compared to language models predicting the next word, Netflix's model uses this method to predict the next viewing choice, considering both immediate past actions and longer-term user patterns.

#### 3. **Handling Metadata and Embeddings:**
   - Each title or interaction isn't just labeled with a unique ID but also packed with **metadata**, like genre, release date, and language. This metadata gets turned into **embeddings**, which are numerical representations capturing the item's properties. New or unseen content uses these metadata embeddings, allowing recommendations even without previous user interactions. For example, a newly released sci-fi film may derive part of its embedding from similar sci-fi titles' features, helping gauge if it's likely to appeal to the user.

### Example:

Let's imagine a typical user scenario:

- **Old Approach:** If a user watched dark comedies, one model might suggest more comedies based on genre, another might suggest titles starring the same actors. These models might not effectively share insights.
  
- **New Approach:**
  - First, the user's behavior is tokenized—watching the entire "Black Mirror" series, pausing to rewatch favorite scenes, and searching for similar dystopian content might become a "sci-fi dystopia enthusiast" token.
  - The system predicts the user's next interest based on all historical tokens, identifying that they might enjoy a "new psychological thriller."
  - A new movie with no past user interactions leverages metadata—its genre, reminiscent of "Black Mirror," boosts its recommendation potential via initial metadata-based embeddings.

Overall, Netflix's foundation model integrates various interactive actions and metadata into a comprehensive system, leveraging insights from large language models to improve recommendation accuracy and adaptability. By focusing on a unified, data-centric strategy, Netflix can effectively manage extensive user data and adapt to new titles and changing user preferences more efficiently.

# Research Paper
passes the whole paper (better suited for smaller reserach papers/articles) <br>
so for long papers refer to other solution in PDF_LLM repo with vector db and embeddings 

## Pdf Reader

In [21]:
class Paper:
    def __init__(self, url):
        self.url = url
        parsed_url = urlparse(url)
        self.text = ""
        self.title = "No title found"

        if parsed_url.scheme in ('http', 'https'):
            response = requests.get(self.url)
            if response.status_code == 200:
                pdf_bytes = BytesIO(response.content)
                reader = PdfReader(pdf_bytes)
                self._extract(reader)
            else:
                print(f"Failed to fetch PDF. Status code: {response.status_code}")
        elif os.path.isfile(url):
            try:
                with open(url, 'rb') as pdf_file:
                    reader = PdfReader(pdf_file)
                    self._extract(reader)
            except Exception as e:
                print(f"Error reading PDF: {e}")
        else:
            print(f"Invalid file path or URL: {url}")

    def _extract(self, reader):
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        self.text = text
        try:
            self.title = reader.metadata.get("/Title", "No title found") or "No title found"
        except Exception:
            self.title = "No title found"

    def __str__(self):
        return f"Paper(title='{self.title[:1000]}...')"


In [22]:
paper = Paper("d:/Projects/sandbox/data/ActionsSpeakLouderthanWords.pdf")
print(paper)

FloatObject (b'0.000000-29802322') invalid; use 0.0 instead


Paper(title='Actions Speak Louder than Words: Trillion-Parameter Sequential Transducers for Generative Recommendations...')


## Summarize

In [23]:
system_prompt_paper_summary = "You are a research assistant. Your job is to go through the provided article in details, understand the purpose, study the methods and techniques employed, arguments made and the conclusion."

In [24]:
def get_user_prompt_paper_summary(paper):
    user_prompt = f"You are looking at a paper titled {paper.title}"
    user_prompt += "\nPlease provide a short summary of this paper in markdown. \
    After the summary extract the following: \
    1) Title and Author of the research paper. \
    2) Year it was published it \
    3) Objective or aim of the research to specify why the research was conducted \
    4) Background or Introduction to explain the need to conduct this research or any topics the readers must have knowledge about \
    5) Type of research/study/experiment to explain what kind of research it is. \
    6) Methods or methodology to explain what the researchers did to conduct the research \
    7) Results and key findings to explain what the researchers found \
    8) Conclusion tells about the conclusions that can be drawn from this research including limitations and future direction \
    The contents of this paper is as follows:\n\n"
    user_prompt += paper.text
    return user_prompt

In [25]:
def summarize_paper(paper_url):
    paper = Paper(paper_url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_paper_summary(paper),
        system_message=system_prompt_paper_summary
    )
    return response

### Test

In [26]:
def display_paper_summary(paper_url):
    summary = summarize_paper(paper_url)
    display(Markdown(summary))

In [28]:
display_paper_summary("d:/Projects/sandbox/data/ActionsSpeakLouderthanWords.pdf")

FloatObject (b'0.000000-29802322') invalid; use 0.0 instead


Using model: gpt-4o
With system prompt: You are a research assistant. Your job is to go th...
With user prompt: You are looking at a paper titled Actions Speak Lo...
Using OpenAI with model: gpt-4o


# Summary of "Actions Speak Louder than Words: Trillion-Parameter Sequential Transducers for Generative Recommendations"

The paper presents a novel approach to recommendation systems, introducing Generative Recommenders (GRs), a new paradigm that formulates recommendation tasks as sequential transduction tasks. By employing a novel architecture called Hierarchical Sequential Transduction Unit (HSTU), GRs address scaling challenges in recommendation systems, allowing for efficient handling of high cardinality, non-stationary streaming data. HSTU outperforms existing state-of-the-art models both in speed and accuracy. The study demonstrates that HSTU-based GRs can be scaled up to 1.5 trillion parameters, significantly enhancing online performance metrics and enabling a simplification in feature processes for large-scale applications.

## Extracted Information

1. **Title and Authors of the Paper**
   - Title: "Actions Speak Louder than Words: Trillion-Parameter Sequential Transducers for Generative Recommendations"
   - Authors: Jiaqi Zhai, Lucy Liao, Xing Liu, Yueming Wang, Rui Li, Xuan Cao, Leon Gao, Zhaojie Gong, Fangda Gu, Michael He, Yinghai Lu, Yu Shi

2. **Year of Publication**
   - 2024

3. **Objective/Aim of the Research**
   - The research was conducted to explore the use of Transformers in improving the scalability and efficiency of recommendation systems by reformulating traditional Deep Learning Recommendation Models (DLRMs) as sequential transduction tasks within a generative modeling framework.

4. **Background/Introduction**
   - Recommendation systems are critical for personalizing user experiences on online platforms. Despite their use of vast amounts of data, traditional DLRMs do not scale effectively with computational resources. Inspired by the success of Transformers in language and vision tasks, the authors propose using generative models for recommendations to address the scale, feature heterogeneity, and computational costs in these systems.

5. **Type of Research**
   - Methodological and applied research focused on designing, implementing, and evaluating a new model architecture for recommendation systems.

6. **Methods/Methodology**
   - The researchers introduced HSTU, a novel architecture that reformulates recommendation tasks as sequential transduction. The architecture builds on a pooling mechanism that combines categorical and numerical features into a unified time series and manages large vocabulary sizes efficiently. It leverages methods like M-FALCON to reduce inference costs and stochastic length mechanisms to manage sequence sparsity.

7. **Results and Key Findings**
   - HSTU outperformed existing models like transformers in both speed and accuracy on large sequence data. It significantly improved metrics in online A/B tests by 12.4% when deployed on platforms with billions of users. The study demonstrates that the architectural and methodological improvements allow GRs to scale linearly with training compute across extensive ranges, validated on synthetic and public datasets.

8. **Conclusion**
   - The paper concludes that by treating user actions as a new modality and using sequentially trained transducers, recommendation systems can achieve considerable efficiency gains and accuracy. The work marks a step toward creating foundation models in recommendation systems akin to LLMs in NLP, which could reduce feature engineering complexity and carbon footprints in model training. Future directions include further refining the generative approach and understanding its broader applications in search and advertisements.

## QnA

In [29]:
system_prompt_paper_qna = "You are a research assistant. Your job is to go through the provided article in details, understand the purpose, study the methods and techniques employed, arguments made and the conclusion."

In [30]:
def get_user_prompt_paper_qna(website, question):
    user_prompt = f"You are looking at a paper titled {paper.title}. The contents of this paper is as follows:\n\n"
    user_prompt += paper.text
    user_prompt += f"\n\nPlease answer the following question related to this paper: {question}"
    return user_prompt

In [31]:
def paper_qna(url, question):
    paper = Paper(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_paper_qna(paper, question),
        system_message=system_prompt_paper_qna
    )
    return response

### Test

In [32]:
def display_paper_answer(url, question):
    answer = paper_qna(url, question)
    display(Markdown(answer))

In [33]:
display_paper_answer("d:/Projects/sandbox/data/ActionsSpeakLouderthanWords.pdf", "This is very new to me - what are hstu? how do they work? how is this different from traditional recommendation systems? what are the features or training data - whats been done to it? what is the algorithmic idea of this process?")

FloatObject (b'0.000000-29802322') invalid; use 0.0 instead


Using model: gpt-4o
With system prompt: You are a research assistant. Your job is to go th...
With user prompt: You are looking at a paper titled Actions Speak Lo...
Using OpenAI with model: gpt-4o


HSTU (Hierarchical Sequential Transduction Unit) is a novel self-attention encoder architecture designed to handle large-scale, high cardinality, non-stationary recommendation data. The key idea behind HSTU is to provide efficient and scalable training and inference for Generative Recommenders through a specialized attention mechanism and architectural design.

### How HSTU Works:
1. **Architecture Design**: HSTU modifies the traditional attention mechanism by introducing pointwise aggregated attention instead of the standard softmax attention. This change helps in capturing both the intensity of user engagements and the relative ordering of items, which is essential for recommendation tasks.

2. **Attention Mechanism**: The pointwise aggregated attention keeps track of the number of prior data points, which is crucial for understanding user preferences, whereas the softmax normalization often loses this context.

3. **Efficiency**: HSTU is designed to be more computationally efficient than standard Transformers, achieving significant speed-ups (5.3x to 15.2x faster for certain sequence lengths). It does so using efficient GPU-based computations, leveraging sparse grouped GEMMs to handle large and sparse input sequences effectively.

4. **Training**: HSTU uses a new training paradigm called "generative training," which reduces the computational complexity traditionally associated with self-attention mechanisms. This is achieved by sampling rates that allow the model to be trained with less compute while still being exposed to the same amount of information.

### Differences from Traditional Recommendation Systems:
- **Sequential Transduction**: Traditional recommender systems often rely on static, handcrafted features and are not built to handle the dynamic nature of user interaction data efficiently. HSTU-based systems treat recommendation tasks as sequential transduction tasks, transforming them into a generative modeling problem.
  
- **Generative Approach**: Instead of the discriminative setting of typical recommenders, which learn to predict user actions or item ranks, this approach models the joint distribution of all user contents and actions sequentially, allowing for richer interaction modeling.

- **Unified Feature Space**: HSTU consolidates and encodes both categorical (sparse) and numerical (dense) features into a unified sequence, thereby replacing the heterogeneous feature approach in traditional models with a more integrated method.

### Features and Training Data:
- **Feature Consolidation**: HSTU merges various categorical features into a sequential format, compressing features that change infrequently and integrating them into the main sequence of user interactions. This is particularly important for capturing long-term user behavior effectively.

- **Numerical Features Handling**: While numerical features frequently change and may not be entirely sequentialized due to computational constraints, HSTU aims to capture their essence by removing them and relying on a more expressive architectural design.

### Algorithmic Idea:
The core algorithmic innovation in HSTU lies in its attention mechanism and its ability to perform computations efficiently by leveraging sparsity and implementing efficient parallel operations. Additionally, M-FALCON (Microbatched-Fast Attention Leveraging Cacheable OperatioNs) is an inference algorithm introduced to efficiently handle large numbers of candidates during the ranking process, further enhancing scalability.

Overall, by reformulating recommendations as a generative task and improving efficiency through architectural and algorithmic innovations, HSTU represents a step forward in building recommendation systems that can scale effectively to meet the demands of modern online platforms.

# Presentations

## PPT Reader

In [129]:
class PowerPoint:
    def __init__(self, path):
        self.path = path
        self.slides = []
        self.text = ""
        self.title = "No title found"

        if os.path.isfile(path):
            try:
                presentation = Presentation(path)
                self._extract(presentation)
                self.title = presentation.core_properties.title or self.slides[0] if self.slides else "No title found"
            except Exception as e:
                print(f"Error reading presentation: {e}")
        else:
            print(f"Invalid file path: {path}")

    def _extract(self, presentation):
        all_text = []
        for slide in presentation.slides:
            slide_text = []
            for shape in slide.shapes:
                if hasattr(shape, "text") and shape.text.strip():
                    slide_text.append(shape.text.strip())
            combined_slide = "\n".join(slide_text)
            self.slides.append(combined_slide)
            all_text.append(combined_slide)
        self.text = "\n\n".join(all_text)

    def __str__(self):
        return f"PresentationFile(title='{self.title[:100]}...', slides={len(self.slides)}, text='{self.text[:1000]}...')"


In [130]:
test_presentation_path = "d:/Projects/sandbox/data/WildfirePrediction-Proposal-Draft.pptx"
presentation = PowerPoint(test_presentation_path)
print(presentation)

PresentationFile(title='PowerPoint Presentation...', slides=10, text='Wildfire Prediction
Proposal

Use cases
Fire occurrence prediction
Area burned and damage prediction
Effect of weather
Severe weather - Lightning
Temperature rise
Effect of human proceses

Market
**To add spending and loss in and due to wildfires in Canada, US and in general
**Existing solutions and limitations

Data
Climate data 
Wildfire data
**Vegetation Data
**Human activity

Model
Proposed - ANN (1)

Model II
Comparison of models for australia - ann still the better choice because of short term memory (5)

Impact
Weather Network - goes with the brand image
Trust - national provider 
Disaster management and urban and commercial planning
Revenue streams
Government - wildfire / Forest department
International UN disaster management
Tracking of carbon footprint etc. for environmental bodies like Environment Canada, UNEP
Private settlements
Wood/paper industries or suppliers
Visibility and smoke level predictions for

## Summarize

In [131]:
system_prompt_presentation_summary = "You are an assistant that analyzes the contents of a presentation \
and provides a short summary in markdown."

In [132]:
def get_user_prompt_presentation_summary(presentation):
    user_prompt = f"You are looking at a presentation titled {presentation.title}"
    user_prompt += "\nThe contents of this presentation is as follows; \
please provide a short summary of this presentation in markdown.\n\n"
    user_prompt += presentation.text
    return user_prompt

In [133]:
def summarize_presentation(url):
    presentation = PowerPoint(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_presentation_summary(presentation),
        system_message=system_prompt_presentation_summary
    )
    return response

### Test

In [134]:
def display_presentation_summary(url):
    summary = summarize_presentation(url)
    display(Markdown(summary))

In [135]:
display_presentation_summary("d:/Projects/sandbox/data/WildfirePrediction-Proposal-Draft.pptx")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a presentation titled PowerPoin...
Using OpenAI with model: gpt-4o


```markdown
# Wildfire Prediction Proposal

### Overview
This presentation outlines a proposal for predicting wildfires, focusing on models that predict fire occurrence, the area burned, and the effects of weather and human processes. The proposal highlights advancements in the use of Artificial Neural Networks (ANNs) for accurate predictions and explores their potential impact on disaster management.

### Use Cases
- **Fire Occurrence Prediction:** Forecasting the likelihood of wildfire occurrences.
- **Damage Prediction:** Estimating the potential area that could be burned.
- **Weather Impact:** Understanding the role of severe weather, such as lightning and temperature rise.
- **Human Processes:** Analyzing human activities that influence wildfires.

### Market
- **Financial Impact:** Pending data on wildfire-related spending and losses in Canada and the US.
- **Existing Solutions:** Discussion on current solutions and their limitations.

### Data Sources
- Climate data
- Wildfire data
- Proposed inclusion of vegetation data and human activity data.

### Model Proposal
- **ANN Model I:** Proposed model leveraging ANNs.
- **Model Comparison:** Comparison with models used in Australia showing the advantages of ANNs due to their effective short-term memory capabilities.

### Impact & Benefits
- Boosts brand image for weather networks.
- Enhances trust as a national provider in disaster management.
- Influences urban and commercial planning, benefiting government agencies, international bodies, and industries.
- Potential revenue streams include government departments, international disaster management organizations, and private sectors such as wood and paper industries.
- Improves visibility and smoke level predictions for public safety and industry operations.

### Costs
- Estimated cost for proof of concept (POC).
- Data acquisition and storage expenses.
- Training costs associated with model development.

### Implementation
Outlined steps for taking the proposed model from concept to application.

### References / Resources
- Review of ML applications in wildfire sciences - NRC (2020) [Read more](https://cdnsciencepub.com/doi/pdf/10.1139/er-2020-0019#refg23)
- Wildland Fire Suppression and Partnerships by the DOI [Learn more](https://www.doi.gov/wildlandfire/suppression)
- Forest fire risk prediction system, DOI: 10.1016/S0957-4174(03)00095-2
```


## QnA

In [136]:
system_prompt_presentation_qna = "You are an assistant that analyzes the contents of a presentation \
and answers questions related to the content of the presentation, ignoring text that might be navigation related. \
Respond in markdown."

In [137]:
def get_user_prompt_presentation_qna(presentation, question):
    user_prompt = f"You are looking at a presentation titled {presentation.title}"
    user_prompt += "\nThe contents of this presentation is as follows \n\n"
    user_prompt += presentation.text
    user_prompt += f"\n\nPlease answer the following question related to this presentation: {question}"
    return user_prompt

In [138]:
def presentation_qna(url, question):
    presentation = PowerPoint(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_website_qna(presentation, question),
        system_message=system_prompt_website_qna
    )
    return response

### Test

In [139]:
def display_presentation_answer(url, question):
    answer = presentation_qna(url, question)
    display(Markdown(answer))

In [142]:
display_presentation_answer("d:/Projects/sandbox/data/WildfirePrediction-Proposal-Draft.pptx", "What is the model and use case proposed?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a website titled PowerPoint Pre...
Using OpenAI with model: gpt-4o


The proposed model for wildfire prediction on the website is an Artificial Neural Network (ANN). The use cases outlined include:

- Fire occurrence prediction
- Area burned and damage prediction
- Effect of weather, specifically severe weather like lightning and temperature rise
- Impact of human processes on wildfire occurrence and behavior

# Videos

## Youtube Transcribe

In [152]:
class YoutubeVideo:
    def __init__(self, url, language='en'):
        """
        Initialize the YoutubeVideo object with the video URL and fetch its transcript.
        """
        self.url = url
        self.video_id = self._extract_video_id(url)
        self.language = language
        self.text = self._get_transcript()
        self.title = f"Video with ID {self.video_id}"  

    def _extract_video_id(self, url):
        """
        Extract the video ID from the given YouTube URL.
        """
        regex = r"(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|\S*\?v=)|(?:youtu\.be\/))([a-zA-Z0-9_-]{11})"
        match = re.match(regex, url)
        if match:
            return match.group(1)
        else:
            raise ValueError("Invalid YouTube URL")

    def _get_transcript(self):
        """
        Fetch the transcript of the video using the YouTubeTranscriptApi.
        """
        try:
            text = YouTubeTranscriptApi.get_transcript(self.video_id, languages=[self.language])
            return " ".join([item['text'] for item in text])
        except Exception as e:
            print(f"Error fetching transcript: {e}")
            return None

    def __str__(self):
        """
        String representation of the YoutubeVideo object.
        """
        return f"YoutubeVideo(title='{self.title}', video_id='{self.video_id}', transcript='{self.text[:100]}...')"

## Summarize

In [153]:
system_prompt_video_summary = "You are an assistant that analyzes the contents of a video \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [154]:
def get_user_prompt_video_summary(video):
    user_prompt = f"You are looking at a video titled {video.title}"
    user_prompt += "\nThe contents of this video are transcribed as follows; \
please provide a short summary of this video in markdown. \n\n"
    user_prompt += video.text
    return user_prompt

In [155]:
def summarize_video(url):
    video = YoutubeVideo(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_video_summary(video),
        system_message=system_prompt_video_summary
    )
    return response

### Test

In [156]:
def display_video_summary(url):
    summary = summarize_video(url)
    display(Markdown(summary))

In [157]:
display_video_summary("https://www.youtube.com/watch?v=8HslUzw35mc")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a video titled Video with ID 8H...
Using OpenAI with model: gpt-4o


This video explores homeopathy, a widely debated alternative medicine. It explains key principles, such as "like cures like," where substances causing symptoms are used to treat them, and "potentization," involving extreme dilution to enhance effectiveness. Criticisms arise due to the lack of scientific evidence supporting homeopathy's efficacy beyond placebo effects. Despite this, homeopathy remains popular, partly due to its personalized care and emphasis on empathy—an area modern medicine could potentially learn from. The video also discusses the financial scale of the homeopathy industry and its impact on public health. The video is produced by Kurzgesagt, announcing their relaunch of a German channel with new content.

## QnA

In [160]:
system_prompt_video_qna = "You are an assistant that analyzes the contents of a video \
and answers questions related to the content of the video transcription \
Respond in markdown."

In [161]:
def get_user_prompt_video_qna(video, question):
    user_prompt = f"You are looking at a video titled {video.title}"
    user_prompt += "\nThe contents of this video is as follows \n\n"
    user_prompt += video.text
    user_prompt += f"\n\nPlease answer the following question related to this video: {question}"
    return user_prompt

In [162]:
def video_qna(url, question):
    video = YoutubeVideo(url)
    response = get_model_response(
        model_name="gpt-4o",
        user_message=get_user_prompt_video_qna(video, question),
        system_message=system_prompt_video_qna
    )
    return response

### Test

In [163]:
def display_video_answer(url, question):
    answer = video_qna(url, question)
    display(Markdown(answer))

In [ ]:
display_video_answer("https://www.youtube.com/watch?v=8HslUzw35mc", "What is the financial share of homeopathy?")

Using model: gpt-4o
With system prompt: You are an assistant that analyzes the contents of...
With user prompt: You are looking at a video titled Video with ID 8H...
Using OpenAI with model: gpt-4o


The video mentions that the global market for homeopathy is expected to reach over $17 billion by 2024. This suggests that homeopathy is a significant industry with substantial financial influence, comparable to other sectors within the pharmaceutical landscape.